**NB :** Ceci est un notebook interactif. Vous pouvez modifier les valeurs ou intéragir avec les graphiques.

Pensez à (re)valider les cellules : **Maj+Entrer**

In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.stats import norm,lognorm
from itertools import permutations
from scipy import linalg,optimize

# Echantillonnage : Dissection de la formule simplifiée de Pierre Gy

La formule simplifiée de Pierre Gy permet d'estimer la variance de l'erreur fondamentale lors d'un échantillonnage.

$\large \sigma_{EF}^2=\left(\frac{1}{M_e}-\frac{1}{M_L}\right) 𝑐 𝑔 𝑙 𝑓 𝑑^3 $

Elle permet de se concentrer sur l'effet :
1. de la masse de l'échantillon prélevé ($M_e$) et 
2. la maille granulométrique prélevée (représentée par le facteur $d$, voir détails ci-dessous)

## Explications détaillées

Dans cette formule, on considère la teneur d'un élément chimique ($t_e$) présents au sein d'une phase minérale ($A$) prise dans une gangue qui regroupe tout le reste ($G$). 

Les blocs ci-dessous vont vous permettre de comprendre la signification et l'impact de chacun de ces facteurs :
* $\sigma_{EF}^2$ est la variance de l'erreur fondamentale (relative).<br>Cette variance relative permet de déduire un écart-type absolu $\sigma=t_A\sqrt{\sigma_{EF}^2}$, en le rapportant à la teneur en minéral $t_A$.
<br>Un interval de confiance de $\pm 2 \sigma$ permet d'encadrer 95% des incertitudes.<br>La valeur réelle a donc 95% de chance d'être dans une gamme $\left[t_A-2\sigma,t_A+2\sigma\right]$
* $M_e$ et $M_L$ représentent les masses de l'échantillon et du lot estimé. En pratique, on considèrera souvent le lot comme très grand par rapport à l'échantillon, donc on peut en faire abstraction : $\left(\frac{1}{M_e}-\frac{1}{M_L}\right)\approx\frac{1}{M_e}$ 
* $c$ : est un facteur de composition : $c =\left(\frac{1-t_A}{t_A}\right)\big[(1-t_A)\rho_A + t_A \rho_G\big]$, où $\rho_A$ est la masse volumique du minéral d'intérêt et $\rho_G$ celle de la gangue
* $g$ : est un facteur de distribution qui prend en compte la répartition de la granulométrie. Ce facteur dépend des dimensions $d_{5\%}$ et $d_{95\%}$.
 * Cette notation $d_{i\%}$ décrit la taille de maille retenant $i\%$ de sa masse. 
 * $d_{5\%}$ est donc une taille assez grande qui ne retient que les 5\%(en masse) des grains les plus gros 
 * $d_{95\%}$ est une taille plus petite qui retient 95% de la masse, donc quasiment l'ensemble des grains des grains, sauf les 5\% de masse représentée par la fraction la plus fine.
* $l$ : est le facteur de libération. Il permet de tenir compte du fait que les minéraux d'intérêts (A) sont au départ pris dans la gangue. Plus les grains sont broyés fins et plus il y a de chance que le minéraux A se retrouvent complètement libéré de leur gangue. 
 * $d_l$ : est la dimension de libération commençante, i.e., la taille à laquelle les particules auront libéré 5% des minéraux A. En d'autres termes, si on réduit des grains à cette taille de manière homogène, 5% de la masse des minéraux A sera libre, 95% sera toujours attaché à des minéraux de gangue.
 * $l$ varie entre 0 et 1 : à 0, la libération est nulle, à 1 elle est totale
 * si $d_{95\%}\leq d_l$, on prendra : $l=1$ (95% des grains doivent être plus gros que $d_l$ donc 5% seulement de A est libéré, mais on considère qu'on a atteint la mailel de libération)
 * si $d_{95\%}> d_l$, on considèrera : $l=\sqrt{\frac{d_l}{d_{95\%}}}$ (moins de 5% des A sont libres la libération n'est donc que trop partielle, ce dont tient compte $l$)
* $f 𝑑^3$ : rend compte du volume des particules : 
 * $f$ est un facteur de forme. C'est le coefficient à appliquer pour passer d'une taille au cube à un volume :
  * pour des particules cubiques : $d$ représente le côté, donc le volume est $d^3$, ce qui correspond à $f=1$
  * pour des particules sphériques : $d$ représente le diamètre, donc le volume est $\frac{4}{3}\pi\left(\frac{d}{2}\right)^3 = \frac{\pi}{6}d^3$, ce qui correspond à $f=\frac{\pi}{6}=0.5236$
  * pour des particules allongées, e.g., l'or alluvial, on pourra prendre $f=0.2$
  * par défaut, si on ne connait pas la forme des particules on considèrera qu'elles sont proches de sphère : $f=0.5$

## Visualisation de l'effet des différents paramètres

### Visualisation de l'effet de $\sigma_{EF}^2$ et des teneurs $t_A$ et $t_e$
$\sigma_{EF}^2$ est la variance de l'erreur fondamentale (relative).
<br>Cette variance relative permet de déduire un écart-type absolu $\sigma=t_A\sqrt{\sigma_{EF}^2}$, en le rapportant à la teneur en minéral $t_A$.

Un interval de confiance de $\pm 2 \sigma$ permet d'encadrer 95% des incertitudes. La valeur réelle a donc 95% de chance d'être dans une gamme $\left[t_A-2\sigma,t_A+2\sigma\right]$

In [2]:
def compute_std(tA,var_EF):
    return tA*np.sqrt(var_EF)
def compute_confidence_range(tA,std,confidence):
    n=norm().isf((1-confidence/100)/2)
    return [tA-n*std,tA+n*std]

In [4]:
def annotate_sigma(dist,n_sigma,ax):
    height = 0.16*(1-(abs(n_sigma)-1)/2)+0.02
    ax.annotate("", xy=(dist.mean()-n_sigma*dist.std(), height), xytext=(dist.mean(), height),
                 arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))
    ax.text(dist.mean()-n_sigma*dist.std()/2, height, r'{}$\sigma$'.format(abs(n_sigma) if abs(n_sigma)>1 else ""),
             {'color': 'black', 'fontsize': 16, 'ha': 'center', 'va': 'center',
              'bbox': dict(boxstyle="round", fc="white", pad=0.2)})

perc={}
perc[1]=68
perc[2]=95
perc[3]=99.7
def annotate_perc(dist,n_sigma,ax):
    height = 0.15*(1-(abs(n_sigma)-1)/2)+0.005
    ax.annotate("", xy=(dist.mean()-n_sigma*dist.std(), height),
                 xytext=(dist.mean()+n_sigma*dist.std(), height),
                 arrowprops=dict(arrowstyle="<->", connectionstyle="arc3"))

    ax.text(dist.mean(), height, "{}%".format(perc[n_sigma]),
             {'color': 'black', 'fontsize': 14, 'ha': 'center', 'va': 'center',
              'bbox': dict(boxstyle="round", fc="white", pad=0.2)})    

@interact(te=widgets.FloatSlider(min=0.001,max=0.999,step=0.001,value=0.1,continuous_update=False),
          tA=widgets.FloatSlider(min=0.001,max=0.999,step=0.001,value=0.1,continuous_update=False),
          EF_perc=widgets.IntSlider(min=1,max=100,step=1,value=10,continuous_update=False),
          confidence=widgets.FloatSlider(min=50,max=99.99,value=95.45,step=0.01,continuous_update=False),
          show_switch=widgets.Dropdown(options=["minéral porteur","élément d'intérêt"],description='Teneur en :'))
def interact_confidence_range(te,tA,EF_perc,confidence,show_switch):
    fig,ax = plt.subplots(1,2)

    N_4_2 = norm(loc=4,scale=2)
    range_min,range_max = [-3,11]
    x= np.linspace(range_min,range_max,100)
    ax[0].plot(x,N_4_2.pdf(x))
    ax[0].axvline(x=N_4_2.mean(),color='r')
    ax[0].axvline(x=N_4_2.mean()-N_4_2.std(),color='r',linestyle="--",ymax=0.9)
    ax[0].axvline(x=N_4_2.mean()+N_4_2.std(),color='r',linestyle="--",ymax=0.9)
    ax[0].axvline(x=N_4_2.mean()-2*N_4_2.std(),color='gray',linestyle="--",ymax=0.5)
    ax[0].axvline(x=N_4_2.mean()+2*N_4_2.std(),color='gray',linestyle="--",ymax=0.5)
    ax[0].axvline(x=N_4_2.mean()-3*N_4_2.std(),color='gray',linestyle="--",ymax=0.125)
    ax[0].axvline(x=N_4_2.mean()+3*N_4_2.std(),color='gray',linestyle="--",ymax=0.125)
    annotate_sigma(N_4_2,-1,ax[0])
    annotate_sigma(N_4_2,1,ax[0])
    annotate_sigma(N_4_2,-2,ax[0])
    annotate_sigma(N_4_2,2,ax[0])
    annotate_sigma(N_4_2,-3,ax[0])
    annotate_sigma(N_4_2,3,ax[0])
    annotate_perc(N_4_2,1,ax[0])
    annotate_perc(N_4_2,2,ax[0])
    annotate_perc(N_4_2,3,ax[0])
    
    ax[0].set_title("Gamme d'incertitude en fonction du nombre d'écart-type")

    tA = tA if show_switch == "minéral porteur" else tA*te
    scale_factor = 1/100
    sigma = tA*EF_perc * scale_factor
    range_min,range_max = compute_confidence_range(tA,sigma,confidence)
    N = norm(loc=tA,scale=sigma)
    x= np.linspace(tA-4*sigma,tA+4*sigma,1000)
    pdf = N.pdf(x)
    ax[1].set_ylim(0,1.05*N.pdf(tA))
    ax[1].plot(x / scale_factor,pdf,label="Distribution de probabilité de "+r"$t_A$")
    ax[1].axvline(x=tA / scale_factor,color='g',linestyle="-",ymax=0.99,label="Estimation")
    ax[1].axvline(x=range_min / scale_factor,color='r',linestyle="--",ymax=0.9,label="Interval de confiance à {}".format(confidence))
    ax[1].axvline(x=range_max / scale_factor,color='r',linestyle="--",ymax=0.9)
    
    handles, labels = ax[1].get_legend_handles_labels()
    handles.append(patches.Patch(color='none', label="Erreur fondamentale (EF) : {}%".format(EF_perc)))
    handles.append(patches.Patch(color='none', label="Variance d'erreur fondamentale : {}={:.3f}".format(r"$\sigma_{EF}2$",(EF_perc/100)**2 )))
    ax[1].legend(handles=handles)

    graph_max = max(0.5 if show_switch == "minéral porteur" else 0.1,1.5*range_max)/scale_factor
    ax[1].set_xlim(0,graph_max)
    ax[1].set_xticks(np.arange(0,graph_max,10 if show_switch == "minéral porteur" else 1))
    ax[1].set_xlabel("Teneur en "+show_switch+" (en %masse)")
    
    ax[1].set_title("Gamme d'incertitude en teneur ({})".format(show_switch))
    
    fig.set_size_inches((16,4))

interactive(children=(FloatSlider(value=0.1, continuous_update=False, description='te', max=0.999, min=0.001, …